In [300]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from xgboost import XGBClassifier
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import plot_importance

In [243]:
df = pd.read_csv('odi_Matches_Data.csv')

In [244]:
df.shape

(4686, 33)

In [245]:
df.columns

Index(['ODI Match No', 'Match ID', 'Match Name', 'Series ID', 'Series Name',
       'Match Date', 'Match Format', 'Team1 ID', 'Team1 Name', 'Team1 Captain',
       'Team1 Runs Scored', 'Team1 Wickets Fell', 'Team1 Extras Rec',
       'Team2 ID', 'Team2 Name', 'Team2 Captain', 'Team2 Runs Scored',
       'Team2 Wickets Fell', 'Team2 Extras Rec', 'Match Venue (Stadium)',
       'Match Venue (City)', 'Match Venue (Country)', 'Umpire 1', 'Umpire 2',
       'Match Referee', 'Toss Winner', 'Toss Winner Choice', 'Match Winner',
       'Match Result Text', 'MOM Player', 'Team1 Playing 11',
       'Team2 Playing 11', 'Debut Players'],
      dtype='object')

In [246]:
df.head()

,ODI Match No,Match ID,Match Name,Series ID,Series Name,Match Date,Match Format,Team1 ID,Team1 Name,Team1 Captain,...,Umpire 2,Match Referee,Toss Winner,Toss Winner Choice,Match Winner,Match Result Text,MOM Player,Team1 Playing 11,Team2 Playing 11,Debut Players
0,488,65425,Australia Vs New Zealand 4Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-07,ODI,2.0,Australia,1572.0,...,RC Bailhache,NaN,Australia,bat,Australia,Australia won by 6 runs,1795.0,"['1767', '1793', '1754', '1572', '1871', '1795...","['1550', '1863', '1861', '1669', '1698', '1846...",[]
1,492,65428,New Zealand Vs Sri Lanka 7Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-12,ODI,5.0,New Zealand,1698.0,...,SG Randell,NaN,Sri Lanka,bowl,Sri Lanka,Sri Lanka won by 4 wickets (with 21 balls rema...,1810.0,"['1777', '1550', '1698', '1669', '1861', '1846...","['1810', '1864', '1789', '1762', '1666', '1664...",[]
2,495,65431,Australia Vs New Zealand 10Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-17,ODI,5.0,New Zealand,1698.0,...,AR Crafter,NaN,Australia,bowl,Australia,Australia won by 5 wickets (with 64 balls rema...,1795.0,"['1550', '1863', '1698', '1669', '1407', '1790...","['1793', '1767', '1773', '1754', '1871', '1795...",['1790']
3,496,65432,Australia Vs Sri Lanka 11Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-19,ODI,8.0,Sri Lanka,1664.0,...,TA Prue,NaN,Australia,bowl,Australia,Australia won by 3 wickets (with 3 balls remai...,1871.0,"['1810', '1864', '1789', '1753', '1762', '1666...","['1767', '1793', '1754', '1871', '1795', '1572...",['1753']
4,508,64326,New Zealand Vs England 3Rd Odi,60882.0,England tour of New Zealand - 1988 (1987/88),1988-03-16,ODI,1.0,England,1543.0,...,SJ Woodward,NaN,New Zealand,bowl,New Zealand,New Zealand won by 7 wickets (with 21 balls re...,1550.0,"['1758', '1629', '1770', '1850', '1543', '1865...","['1550', '1883', '1863', '1882', '1777', '1918...",['1883']


In [247]:
df['Team1 Name'].value_counts()

Australia       551
Pakistan        510
India           491
Sri Lanka       462
New Zealand     411
West Indies     393
England         381
South Africa    333
Zimbabwe        282
Bangladesh      217
Ireland          95
Scotland         84
Afghanistan      81
Kenya            81
Netherlands      52
U.A.E.           50
Canada           43
P.N.G.           32
U.S.A.           27
Namibia          25
Nepal            22
Bermuda          19
Oman             19
Hong Kong        14
Asia XI           4
Africa XI         2
Jersey            2
ICC World XI      1
East Africa       1
Name: Team1 Name, dtype: int64

In [248]:
df.isnull().sum()

ODI Match No               0
Match ID                   0
Match Name                 1
Series ID                  1
Series Name                0
Match Date                 1
Match Format               1
Team1 ID                   1
Team1 Name                 1
Team1 Captain              1
Team1 Runs Scored          6
Team1 Wickets Fell         6
Team1 Extras Rec           6
Team2 ID                   1
Team2 Name                 1
Team2 Captain              1
Team2 Runs Scored        100
Team2 Wickets Fell       100
Team2 Extras Rec         100
Match Venue (Stadium)      1
Match Venue (City)         1
Match Venue (Country)      1
Umpire 1                   1
Umpire 2                   1
Match Referee            752
Toss Winner                1
Toss Winner Choice         4
Match Winner             208
Match Result Text          1
MOM Player               317
Team1 Playing 11           0
Team2 Playing 11           0
Debut Players              0
dtype: int64

In [249]:
df['Match Winner'].unique()

array(['Australia', 'Sri Lanka', 'New Zealand', 'West Indies', 'India',
       'England', nan, 'Pakistan', 'Zimbabwe', 'South Africa', 'Kenya',
       'U.A.E.', 'Bangladesh', 'Canada', 'Netherlands', 'ICC World XI',
       'Africa XI', 'Asia XI', 'Bermuda', 'Ireland', 'Scotland',
       'Afghanistan', 'P.N.G.', 'Hong Kong', 'Nepal', 'Namibia', 'Oman',
       'U.S.A.', 'Jersey'], dtype=object)

In [250]:
df['Team1 Name'].unique()

array(['Australia', 'New Zealand', 'Sri Lanka', 'England', 'India',
       'West Indies', 'East Africa', 'Pakistan', 'Canada', 'Zimbabwe',
       'Bangladesh', 'South Africa', 'U.A.E.', 'Kenya', 'Netherlands',
       'Scotland', 'U.S.A.', 'ICC World XI', 'Africa XI', 'Asia XI',
       'Ireland', 'Bermuda', 'Afghanistan', 'Hong Kong', nan, 'P.N.G.',
       'Nepal', 'Namibia', 'Oman', 'Jersey'], dtype=object)

In [251]:
df2 = df
df2.shape

(4686, 33)

In [252]:
team_names = [
    'Australia',
    'New Zealand',
    'Sri Lanka',
    'England',
    'India',
    'Pakistan',
    'Bangladesh',
    'South Africa',
    'Netherlands',
    'Afghanistan'
]
df3 = pd.DataFrame()

for team in team_names:
    team_data = df2[df2['Team1 Name'] == team]
    df3 = pd.concat([df3, team_data], ignore_index=True)

In [253]:
team_names = [
    'Australia',
    'New Zealand',
    'Sri Lanka',
    'England',
    'India',
    'Pakistan',
    'Bangladesh',
    'South Africa',
    'Netherlands',
    'Afghanistan'
]

df4 = pd.DataFrame()

for team in team_names:
    team_data = df3[df3['Team2 Name'] == team]
    df4 = pd.concat([df4, team_data], ignore_index=True)

In [254]:
df3.shape

(3489, 33)

In [255]:
df4.shape

(2669, 33)

In [256]:
df3.head()

,ODI Match No,Match ID,Match Name,Series ID,Series Name,Match Date,Match Format,Team1 ID,Team1 Name,Team1 Captain,...,Umpire 2,Match Referee,Toss Winner,Toss Winner Choice,Match Winner,Match Result Text,MOM Player,Team1 Playing 11,Team2 Playing 11,Debut Players
0,488,65425,Australia Vs New Zealand 4Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-07,ODI,2.0,Australia,1572.0,...,RC Bailhache,NaN,Australia,bat,Australia,Australia won by 6 runs,1795.0,"['1767', '1793', '1754', '1572', '1871', '1795...","['1550', '1863', '1861', '1669', '1698', '1846...",[]
1,2,64944,England Vs Australia 1St Odi,60784.0,Australia tour of England and Scotland - 1972...,1972-08-24,ODI,2.0,Australia,1243.0,...,AEG Rhodes,NaN,Australia,bat,England,England won by 6 wickets (with 35 balls remain...,1285.0,"['1278', '1291', '1243', '1364', '1394', '1307...","['1228', '1285', '1316', '858', '1334', '1283'...","['1395', '1449', '858', '1285', '1369', '1300'..."
2,4,64946,England Vs Australia 3Rd Odi,60784.0,Australia tour of England and Scotland - 1972...,1972-08-28,ODI,2.0,Australia,1243.0,...,DJ Constant,NaN,England,bowl,England,England won by 2 wickets (with 21 balls remain...,1396.0,"['1278', '1394', '1243', '1364', '1307', '1277...","['1228', '1285', '858', '1316', '1283', '1396'...","['1396', '1411']"
3,11,64151,New Zealand Vs Australia 2Nd Odi,60788.0,Australia tour of New Zealand - 1974 (1973/74),1974-03-31,ODI,2.0,Australia,1243.0,...,FR Goodall,NaN,Australia,bat,Australia,Australia won by 31 runs,NaN,"['1425', '1278', '1243', '1364', '1277', '1215...","['1328', '1456', '1408', '1326', '1311', '1338...","['1425', '1408']"
4,16,64152,Australia Vs England Only Odi,60791.0,England tour of Australia - 1974 (1974/75),1975-01-01,ODI,2.0,Australia,1243.0,...,TF Brooks,NaN,England,bowl,England,England won by 3 wickets (with 23 balls remain...,1243.0,"['1215', '1436', '1243', '1364', '1394', '1277...","['1285', '1430', '1360', '1316', '1403', '1393...","['1424', '1360', '1401', '1359', '1436']"


In [257]:
df4.head()

,ODI Match No,Match ID,Match Name,Series ID,Series Name,Match Date,Match Format,Team1 ID,Team1 Name,Team1 Captain,...,Umpire 2,Match Referee,Toss Winner,Toss Winner Choice,Match Winner,Match Result Text,MOM Player,Team1 Playing 11,Team2 Playing 11,Debut Players
0,495,65431,Australia Vs New Zealand 10Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-17,ODI,5.0,New Zealand,1698.0,...,AR Crafter,NaN,Australia,bowl,Australia,Australia won by 5 wickets (with 64 balls rema...,1795.0,"['1550', '1863', '1698', '1669', '1407', '1790...","['1793', '1767', '1773', '1754', '1871', '1795...",['1790']
1,10,64150,New Zealand Vs Australia 1St Odi,60788.0,Australia tour of New Zealand - 1974 (1973/74),1974-03-30,ODI,5.0,New Zealand,1251.0,...,LHG Harmer,NaN,New Zealand,bat,Australia,Australia won by 7 wickets (with 85 balls rema...,NaN,"['1456', '1328', '1251', '1326', '1311', '1259...","['1215', '1278', '1243', '1364', '1418', '1277...","['1410', '1419', '1423', '1418', '1422', '1402..."
2,98,65303,Australia Vs New Zealand 4Th Match,60812.0,"Benson & Hedges World Series Cup Australia, In...",1980-12-07,ODI,5.0,New Zealand,1311.0,...,RC Bailhache,NaN,New Zealand,bat,Australia,Australia won by 4 wickets (with 16 balls rema...,1572.0,"['1566', '1550', '1421', '1625', '1407', '1311...","['1541', '1572', '1364', '1530', '1277', '1646...",[]
3,107,65311,Australia Vs New Zealand 12Th Match,60812.0,"Benson & Hedges World Series Cup Australia, In...",1981-01-13,ODI,5.0,New Zealand,1444.0,...,MW Johnson,NaN,New Zealand,bat,New Zealand,New Zealand won by 1 run,1550.0,"['1550', '1566', '1444', '1408', '1311', '1421...","['1572', '1547', '1364', '1530', '1277', '1646...",[]
4,111,65315,Australia Vs New Zealand 1St Final,60812.0,"Benson & Hedges World Series Cup Australia, In...",1981-01-29,ODI,5.0,New Zealand,1444.0,...,RC Bailhache,NaN,Australia,bowl,New Zealand,New Zealand won by 78 runs,NaN,"['1550', '1566', '1444', '1421', '1311', '1407...","['1547', '1647', '1361', '1364', '1530', '1572...",['1647']


In [259]:
df4.columns

Index(['ODI Match No', 'Match ID', 'Match Name', 'Series ID', 'Series Name',
       'Match Date', 'Match Format', 'Team1 ID', 'Team1 Name', 'Team1 Captain',
       'Team1 Runs Scored', 'Team1 Wickets Fell', 'Team1 Extras Rec',
       'Team2 ID', 'Team2 Name', 'Team2 Captain', 'Team2 Runs Scored',
       'Team2 Wickets Fell', 'Team2 Extras Rec', 'Match Venue (Stadium)',
       'Match Venue (City)', 'Match Venue (Country)', 'Umpire 1', 'Umpire 2',
       'Match Referee', 'Toss Winner', 'Toss Winner Choice', 'Match Winner',
       'Match Result Text', 'MOM Player', 'Team1 Playing 11',
       'Team2 Playing 11', 'Debut Players'],
      dtype='object')

In [260]:
df4 = df4[['Team1 Name','Team2 Name','Match Venue (Stadium)',
       'Match Venue (City)', 'Match Venue (Country)',
       'Match Winner','Team1 Playing 11','Team2 Playing 11']]

In [261]:
df4.shape

(2669, 8)

In [262]:
df4.head()

,Team1 Name,Team2 Name,Match Venue (Stadium),Match Venue (City),Match Venue (Country),Match Winner,Team1 Playing 11,Team2 Playing 11
0,New Zealand,Australia,Brisbane Cricket Ground,Brisbane,Australia,Australia,"['1550', '1863', '1698', '1669', '1407', '1790...","['1793', '1767', '1773', '1754', '1871', '1795..."
1,New Zealand,Australia,Carisbrook,Dunedin,New Zealand,Australia,"['1456', '1328', '1251', '1326', '1311', '1259...","['1215', '1278', '1243', '1364', '1418', '1277..."
2,New Zealand,Australia,Melbourne Cricket Ground,Melbourne,Australia,Australia,"['1566', '1550', '1421', '1625', '1407', '1311...","['1541', '1572', '1364', '1530', '1277', '1646..."
3,New Zealand,Australia,Sydney Cricket Ground,Sydney,Australia,New Zealand,"['1550', '1566', '1444', '1408', '1311', '1421...","['1572', '1547', '1364', '1530', '1277', '1646..."
4,New Zealand,Australia,Sydney Cricket Ground,Sydney,Australia,New Zealand,"['1550', '1566', '1444', '1421', '1311', '1407...","['1547', '1647', '1361', '1364', '1530', '1572..."


In [263]:
df4.shape

(2669, 8)

In [286]:
df4['Team1 Name'] = df4['Team1 Name'].replace({'Australia': 1})
df4['Team1 Name'] = df4['Team1 Name'].replace({'New Zealand': 2})
df4['Team1 Name'] = df4['Team1 Name'].replace({'Sri Lanka': 3})
df4['Team1 Name'] = df4['Team1 Name'].replace({'England': 4})
df4['Team1 Name'] = df4['Team1 Name'].replace({'India': 5})
df4['Team1 Name'] = df4['Team1 Name'].replace({'Pakistan': 6})
df4['Team1 Name'] = df4['Team1 Name'].replace({'Bangladesh': 7})
df4['Team1 Name'] = df4['Team1 Name'].replace({'South Africa': 8})
df4['Team1 Name'] = df4['Team1 Name'].replace({'Netherlands': 9})
df4['Team1 Name'] = df4['Team1 Name'].replace({'Afghanistan': 10})

In [288]:
df4['Team2 Name'] = df4['Team2 Name'].replace({'Australia': 1})
df4['Team2 Name'] = df4['Team2 Name'].replace({'New Zealand': 2})
df4['Team2 Name'] = df4['Team2 Name'].replace({'Sri Lanka': 3})
df4['Team2 Name'] = df4['Team2 Name'].replace({'England': 4})
df4['Team2 Name'] = df4['Team2 Name'].replace({'India': 5})
df4['Team2 Name'] = df4['Team2 Name'].replace({'Pakistan': 6})
df4['Team2 Name'] = df4['Team2 Name'].replace({'Bangladesh': 7})
df4['Team2 Name'] = df4['Team2 Name'].replace({'South Africa': 8})
df4['Team2 Name'] = df4['Team2 Name'].replace({'Netherlands': 9})
df4['Team2 Name'] = df4['Team2 Name'].replace({'Afghanistan': 10})

In [292]:
df4['Match Winner'] = df4['Match Winner'].replace({'Australia': 1})
df4['Match Winner'] = df4['Match Winner'].replace({'New Zealand': 2})
df4['Match Winner'] = df4['Match Winner'].replace({'Sri Lanka': 3})
df4['Match Winner'] = df4['Match Winner'].replace({'England': 4})
df4['Match Winner'] = df4['Match Winner'].replace({'India': 5})
df4['Match Winner'] = df4['Match Winner'].replace({'Pakistan': 6})
df4['Match Winner'] = df4['Match Winner'].replace({'Bangladesh': 7})
df4['Match Winner'] = df4['Match Winner'].replace({'South Africa': 8})
df4['Match Winner'] = df4['Match Winner'].replace({'Netherlands': 9})
df4['Match Winner'] = df4['Match Winner'].replace({'Afghanistan': 10})

In [289]:
df4.head()

,Team1 Name,Team2 Name,Match Venue (Stadium),Match Venue (City),Match Venue (Country),Match Winner,Team1 Playing 11,Team2 Playing 11
0,2,1,Brisbane Cricket Ground,Brisbane,Australia,Australia,"['1550', '1863', '1698', '1669', '1407', '1790...","['1793', '1767', '1773', '1754', '1871', '1795..."
1,2,1,Carisbrook,Dunedin,New Zealand,Australia,"['1456', '1328', '1251', '1326', '1311', '1259...","['1215', '1278', '1243', '1364', '1418', '1277..."
2,2,1,Melbourne Cricket Ground,Melbourne,Australia,Australia,"['1566', '1550', '1421', '1625', '1407', '1311...","['1541', '1572', '1364', '1530', '1277', '1646..."
3,2,1,Sydney Cricket Ground,Sydney,Australia,New Zealand,"['1550', '1566', '1444', '1408', '1311', '1421...","['1572', '1547', '1364', '1530', '1277', '1646..."
4,2,1,Sydney Cricket Ground,Sydney,Australia,New Zealand,"['1550', '1566', '1444', '1421', '1311', '1407...","['1547', '1647', '1361', '1364', '1530', '1572..."


In [141]:
# df_ohe = pd.get_dummies(data=df4 ,columns=['Team1 Name','Team2 Name'])

In [142]:
# df_ohe.head()

,Match Venue (Stadium),Match Venue (City),Match Venue (Country),Match Winner,Team1 Playing 11,Team2 Playing 11,Team1 Name_Afghanistan,Team1 Name_Australia,Team1 Name_Bangladesh,Team1 Name_England,...,Team2 Name_Afghanistan,Team2 Name_Australia,Team2 Name_Bangladesh,Team2 Name_England,Team2 Name_India,Team2 Name_Netherlands,Team2 Name_New Zealand,Team2 Name_Pakistan,Team2 Name_South Africa,Team2 Name_Sri Lanka
0,Brisbane Cricket Ground,Brisbane,Australia,Australia,"['1550', '1863', '1698', '1669', '1407', '1790...","['1793', '1767', '1773', '1754', '1871', '1795...",0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,Carisbrook,Dunedin,New Zealand,Australia,"['1456', '1328', '1251', '1326', '1311', '1259...","['1215', '1278', '1243', '1364', '1418', '1277...",0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Melbourne Cricket Ground,Melbourne,Australia,Australia,"['1566', '1550', '1421', '1625', '1407', '1311...","['1541', '1572', '1364', '1530', '1277', '1646...",0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Sydney Cricket Ground,Sydney,Australia,New Zealand,"['1550', '1566', '1444', '1408', '1311', '1421...","['1572', '1547', '1364', '1530', '1277', '1646...",0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Sydney Cricket Ground,Sydney,Australia,New Zealand,"['1550', '1566', '1444', '1421', '1311', '1407...","['1547', '1647', '1361', '1364', '1530', '1572...",0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [143]:
# df_ohe['Team_Afghanistan'] = df_ohe['Team1 Name_Afghanistan'] + df_ohe['Team2 Name_Afghanistan']
# df_ohe['Team_Australia'] = df_ohe['Team1 Name_Australia'] + df_ohe['Team2 Name_Australia']
# df_ohe['Team_Bangladesh'] = df_ohe['Team1 Name_Bangladesh'] + df_ohe['Team2 Name_Bangladesh']
# df_ohe['Team_England'] = df_ohe['Team1 Name_England'] + df_ohe['Team1 Name_England']
# df_ohe['Team_India'] = df_ohe['Team1 Name_India'] + df_ohe['Team1 Name_India']
# df_ohe['Team_Pakistan'] = df_ohe['Team1 Name_Pakistan'] + df_ohe['Team2 Name_Pakistan']
# df_ohe['Team_South Africa'] = df_ohe['Team1 Name_South Africa'] + df_ohe['Team2 Name_South Africa']
# df_ohe['Team_Netherlands'] = df_ohe['Team1 Name_Netherlands'] + df_ohe['Team2 Name_Netherlands']
# df_ohe['Team_Sri Lanka'] = df_ohe['Team1 Name_Sri Lanka'] + df_ohe['Team2 Name_Sri Lanka']
# df_ohe['Team_New Zealand'] = df_ohe['Team1 Name_New Zealand'] + df_ohe['Team2 Name_New Zealand']

In [144]:
# df_ohe.columns

Index(['Match Venue (Stadium)', 'Match Venue (City)', 'Match Venue (Country)',
       'Match Winner', 'Team1 Playing 11', 'Team2 Playing 11',
       'Team1 Name_Afghanistan', 'Team1 Name_Australia',
       'Team1 Name_Bangladesh', 'Team1 Name_England', 'Team1 Name_India',
       'Team1 Name_Netherlands', 'Team1 Name_New Zealand',
       'Team1 Name_Pakistan', 'Team1 Name_South Africa',
       'Team1 Name_Sri Lanka', 'Team2 Name_Afghanistan',
       'Team2 Name_Australia', 'Team2 Name_Bangladesh', 'Team2 Name_England',
       'Team2 Name_India', 'Team2 Name_Netherlands', 'Team2 Name_New Zealand',
       'Team2 Name_Pakistan', 'Team2 Name_South Africa',
       'Team2 Name_Sri Lanka', 'Team_Afghanistan', 'Team_Australia',
       'Team_Bangladesh', 'Team_England', 'Team_India', 'Team_Pakistan',
       'Team_South Africa', 'Team_Netherlands', 'Team_Sri Lanka',
       'Team_New Zealand'],
      dtype='object')

In [145]:
# df_ohe_final = df_ohe[['Team_Afghanistan', 'Team_Australia',
#        'Team_Bangladesh', 'Team_England', 'Team_India', 'Team_Pakistan',
#        'Team_South Africa', 'Team_Netherlands', 'Team_Sri Lanka',
#        'Team_New Zealand','Match Venue (Stadium)', 'Match Venue (City)', 'Match Venue (Country)',
#        'Match Winner', 'Team1 Playing 11', 'Team2 Playing 11']]

In [146]:
# df_ohe_final.shape

(2669, 16)

In [147]:
# df_ohe_final.head()

,Team_Afghanistan,Team_Australia,Team_Bangladesh,Team_England,Team_India,Team_Pakistan,Team_South Africa,Team_Netherlands,Team_Sri Lanka,Team_New Zealand,Match Venue (Stadium),Match Venue (City),Match Venue (Country),Match Winner,Team1 Playing 11,Team2 Playing 11
0,0,1,0,0,0,0,0,0,0,1,Brisbane Cricket Ground,Brisbane,Australia,Australia,"['1550', '1863', '1698', '1669', '1407', '1790...","['1793', '1767', '1773', '1754', '1871', '1795..."
1,0,1,0,0,0,0,0,0,0,1,Carisbrook,Dunedin,New Zealand,Australia,"['1456', '1328', '1251', '1326', '1311', '1259...","['1215', '1278', '1243', '1364', '1418', '1277..."
2,0,1,0,0,0,0,0,0,0,1,Melbourne Cricket Ground,Melbourne,Australia,Australia,"['1566', '1550', '1421', '1625', '1407', '1311...","['1541', '1572', '1364', '1530', '1277', '1646..."
3,0,1,0,0,0,0,0,0,0,1,Sydney Cricket Ground,Sydney,Australia,New Zealand,"['1550', '1566', '1444', '1408', '1311', '1421...","['1572', '1547', '1364', '1530', '1277', '1646..."
4,0,1,0,0,0,0,0,0,0,1,Sydney Cricket Ground,Sydney,Australia,New Zealand,"['1550', '1566', '1444', '1421', '1311', '1407...","['1547', '1647', '1361', '1364', '1530', '1572..."


In [290]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

column_name = [
    'Match Venue (Stadium)',
    'Match Venue (City)',
    'Match Venue (Country)'
]
for name in column_name:
    df4[column_name] = label_encoder.fit_transform(column_name)

In [293]:
df4

,Team1 Name,Team2 Name,Match Venue (Stadium),Match Venue (City),Match Venue (Country),Match Winner,Team1 Playing 11,Team2 Playing 11
0,2,1,2,0,1,1.0,"['1550', '1863', '1698', '1669', '1407', '1790...","['1793', '1767', '1773', '1754', '1871', '1795..."
1,2,1,2,0,1,1.0,"['1456', '1328', '1251', '1326', '1311', '1259...","['1215', '1278', '1243', '1364', '1418', '1277..."
2,2,1,2,0,1,1.0,"['1566', '1550', '1421', '1625', '1407', '1311...","['1541', '1572', '1364', '1530', '1277', '1646..."
3,2,1,2,0,1,2.0,"['1550', '1566', '1444', '1408', '1311', '1421...","['1572', '1547', '1364', '1530', '1277', '1646..."
4,2,1,2,0,1,2.0,"['1550', '1566', '1444', '1421', '1311', '1407...","['1547', '1647', '1361', '1364', '1530', '1572..."
...,...,...,...,...,...,...,...,...
2664,9,10,2,0,1,9.0,"['48654', '49443', '48655', '2260', '6362', '4...","['53606', '53604', '60436', '53602', '53809', ..."
2665,9,10,2,0,1,10.0,"['48654', '49443', '48655', '6362', '45358', '...","['53606', '60436', '53809', '53602', '53605', ..."
2666,9,10,2,0,1,10.0,"['48655', '6362', '48736', '48111', '2260', '4...","['53767', '53606', '52976', '60436', '53602', ..."
2667,9,10,2,0,1,10.0,"['48655', '48654', '48736', '45001', '2260', '...","['53112', '52976', '60436', '53602', '53809', ..."


In [294]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split


In [295]:
X = df.drop(columns=['Match Winner'], axis=1)
y = df['Match Winner']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [303]:
y_train = y_train.astype(int)

ValueError: invalid literal for int() with base 10: 'England'

In [302]:
model = XGBClassifier()
model.fit(X_train, y_train)

TypeError: '<' not supported between instances of 'str' and 'float'

In [297]:
param_grid = {
    'alpha': [0.0001, 0.001, 0.01],
    'loss': ['hinge', 'log', 'perceptron'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000, 3000]
}

grid_search = GridSearchCV(sgd_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_sgd_classifier = grid_search.best_estimator_


TypeError: '<' not supported between instances of 'str' and 'float'